Hash Tables
============

A hash table is a data structure which allows a user to look up a value within the data structure using a specific key. The value is stored at an index which is calculated by hashing the key. By calculating the hash based on the key, the index of the value is unique, and the index may be calculated again for lookup simply by passing the key to a hash function. This effectively allows a user to access the contents of the hash table without iterating over each value, meaning that the value may be retrieved much more quickly than if it were stored in an array alone. 

Hash tables come with their own set of problems to solve, however. The system falls apart if the index of one key/value pair is equivalent to the index of a completely different key/value pair. This problem, known as Collision, may be solved in a number of ways, though usually a combination of approaches is used. The hash function must generate sufficiently different indices for keys that are similar, and the hash table must have a way to deal with collisions when they occur.

For my implementation, I've chosen to use a collision algorithm known commonly as the "separate chaining" algorithm. In this implementation, the hash table actually stores a linked list at each index rather than the exact value on its own. If a collision arises, the colliding value is added on to the beginning of the linked list at that index. 

For my hash generation, I've chosen to take the key as input, convert each character in the key to its ASCII value, make each ascii value a digit (or series of digits) in a new number, and then perform mod division using a large prime number. This way, the index produced by hashing may be calculated quickly and the large prime number ensures that each index is sufficiently unique. 

As far as the O() behavior goes, accessing an index where there hasn't been a collision should take constant time, at least as much time as it takes to hash the index and then return the value at the hashed index. In the worst case, where the index has collided with another index that has been previously occupied, then the O() behavior of that retrieval would be closer to O(n), where n is the number of steps down the linked list to reach the desired value. I find the separate chaining algorithm to be more desirable than linear probing for this reason, as the worst case calculation for the linear probing algorithm, also O(n), is considerably more likely, and more difficult to handle as calculating the index of a particular entry that has been placed via linear probing requires extra calculation beyond the hashing function. 

In reviewing for this question, I read Skiena's Algorithm Design Manual, specifically section 3.3 on dictionaries (hash tables are dictionaries with hashing, essentially). I also read [these slides](https://study.cs50.net/hashtables) to get an idea of what kinds of collision algorithms were possible. 

In [130]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [131]:
def hash(key):
    """
    this function takes a string as input and returns an integer representation
    of that string.
    """
    result = ''
    for letter in key:
        result += str(ord(letter))
    result = int(result) % 7919
    return result

# slightly different permutations should have largely different indices.
print(hash("the"))
print(hash("whale"))
print(hash("what"))
print(hash("when"))
print(hash("why")) 
print(hash("who"))

3642
570
4356
5005
2361
2351


In [132]:
class LinkedList:
    def __init__(self, key, value, next=None):
        #typically there would only be one value, 
        #but I figured that, given the purpose of this
        #linked list, having two values made sense. 
        self.key = key
        self.value = value
        self.next = next

class Dictionary:
    def __init__(self, size=100):
        self.hashtable = []
        self.size = size
        for i in range(self.size):
            self.hashtable.append([])
            
    def insert(self, key, value):
        """
        this function takes a key (string) and value (any type) as input
        and appends that key value pair to self.hashtable. 
        """
        index = hash(key)%self.size # limit to size of the table
        if self.hashtable[index] == []:
            self.hashtable[index] = LinkedList(key,value) #storing both key and value for traversal
        else:
            self.hashtable[index] = LinkedList(key,value, next=self.hashtable[index]) #newest at top
    
    def modifyValue(self, key, value):
        """
        this function takes a key and value as input, finds the entry in 
        hashtable that corresponds to the key, and then changes the value
        of the key value pair to the new value. 
        """
        index = hash(key)%self.size
        if self.hashtable[index].key == key:
            self.hashtable[index].value = value
        else:
            modified = self.modifyValueLL(key, value, self.hashtable[index].next)
    
    def modifyValueLL(self, key, value, linkedList):
        """
        helper function to recursively traverse linked list and change value of
        key value pairs. 
        """
        if linkedList.key == key:
            linkedList.value = value
            return linkedList
        elif linkedList.key != key and linkedList.next != None:
            self.modifyValueLL(key, value, linkedList.next)
        else:
            return None # only if key isn't there
            
    def search(self, key):
        """
        this function takes a key (string) as input and searches the hash table
        for a matching value. if the value exists, that value is returned. else;
        return None. 
        """
        index = hash(key)%self.size
        currentNode = self.hashtable[index] # this should be a linked list or an empty list
        while True:
            if currentNode == []:
                return None
            elif currentNode.key == key:
                return currentNode.value
            elif currentNode.key != key and currentNode.next != None:
                currentNode = currentNode.next
                continue
            else:
                return None

In [133]:
mydict = Dictionary()
mydict.insert("name", "Nick")
mydict.insert("age", "22")
print(mydict.search("age"))
mydict.modifyValue("age", "30")
print(mydict.search("age")) # this is going to return whichever one it finds first (so most recently added)
print(mydict.search("name"))

22
30
Nick


In [134]:
import time
mobyDickWordCount = Dictionary(size=10000) # there could be a lot of different words! taking no chances. 

with open('mobydick.txt', 'r') as file:
    text = file.read()

    #cleaning up the file a bit (this isn't perfect), as you'll see, but close enough for me.
words = text.replace('\n', '')
words = words.replace('\ufeff', '')
words = words.split(' ')

for word in words:
    if word != '':
        inDict = mobyDickWordCount.search(word)
        if inDict != None:
            mobyDickWordCount.modifyValue(word, inDict + 1)
        else:
            mobyDickWordCount.insert(word, 1)
# I tried to work out a histogram using matplotlib, but the sheer size of this data made it
# more complicated than i thought necessary
# so instead, i've printed each word alongside the number of times it appears in the text. 
word4Hist = []
wordcount = []
for word in words:
    if word != '' and word not in word4Hist: 
        word4Hist.append(word)
        wordcount.append(mobyDickWordCount.search(word))
print(len(wordcount))
for i in range(len(wordcount)):
    print(f"{wordcount[i]}: {word4Hist[i]}")

44476
482: The
62: Project
15: Gutenberg
1: EBook
5887: of
69: Moby
8: Dick;
671: or
33: Whale,
998: by
1: HermanMelvilleThis
3: eBook
1399: is
1220: for
11792: the
31: use
4: anyone
7: anywhere
1108: at
387: no
2: cost
5125: and
1448: with
2: almostno
2: restrictions
3: whatsoever.
37: You
205: may
13: copy
211: it,
58: give
1368: it
99: away
2: re-use
2: itunder
25: terms
7: License
5: included
2: thiseBook
4: online
1: www.gutenberg.orgTitle:
1: WhaleAuthor:
3: Herman
1: MelvilleRelease
1: Date:
2: December
1: 25,
1: 2008
1: [EBook
1: #2701]
1: Last
1: Updated:
1: 3,2017Language:
1: EnglishCharacter
62: set
1: encoding:
1: UTF-8***
1: START
14: OF
8: THIS
4: PROJECT
3: GUTENBERG
2: EBOOK
2: MOBY
2: DICK;
8: OR
13: THE
2: WHALE
1: ***Produced
4: Daniel
2: Lazarus,
2: Jonesey,
2: David
1: WidgerMOBY-DICK;or,
1: WHALE.By
1: MelvilleCONTENTSETYMOLOGY.EXTRACTS
3: (Supplied
3931: a
1: Sub-Sub-Librarian).CHAPTER
4: 1.
1: Loomings.CHAPTER
3: 2.
1: Carpet-Bag.CHAPTER
3: 3.
1: Spouter-Inn.CHA

5: prey,
5: chance,
1: fry,
4: Which
3: gaping
20: jaws
7: mistake
1: way.”
1: —_Dryden’s
1: Annus
1: Mirabilis_.
19: floating
13: stern
76: ship,
489: they
30: cut
143: off
58: head,
9: tow
111: boat
49: near
7: come;
1: aground
13: twelve
2: thirteen
74: feet
1: water.”
3: —_Thomas
1: Edge’s
2: Ten
3: Voyages
1: Spitzbergen,
1: Purchas_.
134: way
94: saw
4: sporting
14: ocean,
1: wantonness
1: fuzzing
61: water
5: pipes
1: vents,
16: nature
31: placed
1: shoulders.”
1: Herbert’s
2: Asia
1: Africa.
1: Harris
2: Coll_.
1: “Here
285: such
2: troops
15: forced
8: proceed
9: deal
1: caution
18: fear
159: should
34: run
196: ship
1: —_Schouten’s
1: Sixth
1: Circumnavigation_.
2: “We
43: sail
1: Elbe,
31: wind
1: N.E.
1: Jonas-in-the-Whale....
23: Some
21: can’t
52: open
1: fable....
13: frequently
3: climb
15: masts
152: whale,
162: first
1: discoverer
1: ducat
1: pains....
35: told
1: Shetland,
34: above
1: barrel
1: herrings
1: belly....
26: harpooneers
33: caught
99: once
3: Spitzbergen

1: Fuego),
1: “over
2: beech
8: tree
5: extended
1: branches.”
1: —_Darwin’s
1: Naturalist_.
1: “‘Stern
1: all!’
5: exclaimed
21: mate,
39: turning
4: distended
3: threatening
38: instant
1: destruction;—‘Stern
39: all,
1: lives!’”
1: —_Wharton
1: Killer_.
2: cheery,
478: my
4: lads,
14: While
11: bold
36: harpooneer
1: whale!”
1: —_Nantucket
1: Song_.
1: “Oh,
10: rare
3: mid
11: storm
9: gale
21: home
1: giant
4: might,
4: right,
4: boundless
1: —_Whale
1: Song_.CHAPTER
1: Loomings.Call
2: Ishmael.
1: ago—never
41: mind
1: precisely—havinglittle
7: money
3: purse,
30: particular
10: interest
2: meon
3: shore,
104: thought
15: watery
5: partof
8: driving
1: spleen
1: andregulating
1: circulation.
1: Whenever
6: growing
7: grim
4: aboutthe
3: mouth;
11: whenever
4: damp,
1: drizzly
1: November
6: soul;
1: wheneverI
8: involuntarily
6: pausing
17: coffin
1: warehouses,
1: andbringing
12: rear
2: funeral
1: meet;
36: especially
1: whenevermy
1: hypos
78: get
24: upper
126: me,
1: moralpri

1: soles
6: boots
1: mostmiserable
2: plight.
1: Too
1: jolly,
89: again
1: onemoment
19: watch
36: broad
2: glare
8: sounds
1: tinkling
2: glasses
2: within.
39: on,
7: last;
1: don’tyou
4: hear?
2: door;
4: patched
1: arestopping
13: way.
1: went.
3: instinct
17: followed
1: streetsthat
38: took
1: waterward,
92: there,
5: doubtless,
1: cheapest,
4: notthe
1: cheeriest
1: inns.Such
1: streets!
4: blocks
1: blackness,
1: houses,
2: hand,and
1: candle,
5: candle
1: tomb.
1: Atthis
17: hour
186: last
84: day
1: week,
8: quarter
9: town
12: proved
1: deserted.
5: presently
2: smoky
1: lightproceeding
5: low,
1: building,
19: door
1: stoodinvitingly
3: open.
1: careless
21: look,
12: meant
1: theuses
1: public;
60: so,
1: entering,
1: stumbleover
1: ash-box
1: porch.
8: Ha!
3: ha,
1: flyingparticles
3: choked
5: ashes
1: city,Gomorrah?
1: Crossed
1: Harpoons,”
1: Sword-Fish?”—this,
1: thenmust
1: Trap.”
7: However,
9: picked
1: andhearing
8: loud
5: within,
13: pushed
7: opened
2: second,

1: apartment,
9: hammock,
1: andcover
1: skin.The
4: pondered
1: abominated
3: thethought
10: sleeping
22: fair
3: presume
1: aharpooneer,
3: linen
1: woollen,
1: tidiest,
19: certainly
25: none
1: finest.
2: twitch
1: over.Besides,
20: getting
4: late,
8: ought
1: homeand
1: bedwards.
1: Suppose
3: tumble
1: atmidnight—how
5: vile
10: hole
1: coming?“Landlord!
25: I’ve
9: changed
1: harpooneer.—I
1: shan’t
2: sleepwith
51: I’ll
1: here.”“Just
1: please;
22: I’m
6: sorry
19: spare
1: tablecloth
1: amattress,
2: plaguy
3: rough
1: here”—feeling
1: knots
1: andnotches.
5: wait
4: bit,
1: Skrimshander;
42: got
5: carpenter’s
1: planethere
1: bar—wait,
4: snug
1: enough.”
1: sayinghe
1: procured
1: plane;
1: silk
2: handkerchief
1: dustingthe
2: bench,
4: vigorously
1: planing
1: grinninglike
1: ape.
4: shavings
21: flew
3: left;
1: theplane-iron
1: bump
2: indestructible
1: knot.
1: wasnear
1: spraining
2: wrist,
4: heaven’s
8: sake
1: quit—thebed
17: soft
1: suit
1: planingin
1: eider
7:

...

1: thyselfunbegotten;
2: beginning,
1: thyselfunbegun.
1: thouomnipotent.
1: unsuffusing
1: clearspirit,
1: creativenessmechanical.
2: self,
1: dimlysee
1: foundling
1: hastthy
1: incommunicable
1: riddle,
1: unparticipated
2: grief.
1: withhaughty
1: agony,
1: sire.
1: Leap!
2: sky!
1: leapwith
1: defyinglyI
1: thee!”“The
1: man!”Ahab’s
1: lashedin
1: whale-boat’sbow;
1: leathersheath
1: alevelled
1: therelike
1: serpent’s
1: arm—“God,
1: forbear!
1: voyage!
1: begun,
1: illcontinued;
1: afair
1: homewards,
1: this.”Overhearing
1: panic-stricken
1: thebraces—though
1: aghastmate’s
1: cry.
1: Butdashing
4: links
1: theburning
1: torch
1: totransfix
1: end.Petrified
1: shrinking
1: dartthat
1: dismay,
1: spoke:—“All
1: binding
1: andheart,
1: bound.
1: yemay
1: beats;
1: fear!”
1: extinguished
1: theflame.As
1: hurricane
1: sweeps
1: neighborhood
1: ofsome
1: lone,
1: elm,
1: itso
1: unsafe,
1: forthunderbolts;
1: didrun
1: dismay.CHAPTER
1: Watch._Ahab
1: him._“We
3: main-top-sail
1: w

1: thecraters
2: grave-digger
1: sings,
1: inhand.
1: never?”“Sing,
1: sing?
1: music
2: mallet
1: Harkto
1: it.”“Aye,
1: sounding-board;
1: sounding-board
1: this—there’s
1: beneath.
1: Andyet,
1: Carpenter.Hast
1: bier,
1: gate,
1: in?“Faith,
1: I’ve——”“Faith?
1: that?”“Why,
1: exclamation-like—that’s
1: all,sir.”“Um,
1: um;
1: on.”“I
1: that——”“Art
1: silk-worm?
1: thyself?Look
1: bosom!
1: Despatch!
1: traps
1: sight.”“He
1: hotlatitudes.
1: Albemarle,
1: theGallipagos,
2: middle.
1: alwaysunder
1: Line—fiery
1: way—come,
1: oakum;quick.
1: theprofessor
2: musical
1: glasses—tap,
1: tap!”(_Ahab
1: himself_.)“There’s
1: greyheaded
1: woodpecker
1: tappingthe
1: tree!
1: Blind
1: envied
1: tow-lines.
1: wag,that
1: fellow.
1: Rat-tat!
1: seconds
1: tick!
1: immaterial
1: allmaterials!
1: imponderable
1: thoughts?
1: Herenow’s
1: hap,
1: theexpressive
1: Alife-buoy
1: further?
1: somespiritual
1: immortality-preserver!I’ll
1: no.
1: earth,that
1: theoretic
1: uncertaintwilight
1: Carp

1: jets.
1: stunsails!
1: Downtop-gallant-sails!
1: stayon
1: Helm
1: Luff,
1: luff
1: point!
1: So;steady,
1: steady!
1: water!
1: Allready
1: Lower
1: Starbuck;lower,
1: lower,—quick,
1: quicker!”
1: deck.“He
1: “right
1: fromus;
1: yet.”“Be
1: Hard
1: helm!—brace
1: up!Shiver
1: her!—shiver
1: her!—So;
1: Boats,
1: boats!”Soon
1: dropped;
1: boat-sailsset—all
1: plying;
1: onset.
1: death-glimmer
1: upFedallah’s
1: gnawed
1: mouth.Like
1: nautilus
2: prows
1: sped
1: sea;but
1: grewstill
1: smooth;
1: anoon-meadow,
1: huntercame
1: unsuspecting
1: dazzlinghump
1: isolatedthing,
1: fleecy,greenish
1: slightlyprojecting
1: beyond.
1: Turkish-ruggedwaters,
1: milkyforehead,
1: playfully
1: andbehind,
1: interchangeably
1: movingvalley
1: anddanced
1: ofhundreds
1: feathering
2: alternate
1: theirfitful
1: flight;
1: flag-staff
1: hullof
1: argosy,
1: projectedfrom
1: ofsoft-toed
1: hovering,
1: skimming
1: tailfeathers
2: streaming
1: pennons.A
1: joyousness—a
1: Jupiter
1: withravishe

1: top-maul!
1: there’stime
1: wink
1: sand-hills
2: same!—the
1: asoft
1: leewardings!
1: leadsomewhere—to
1: thepalms.
1: Leeward!
1: then;the
1: bitterer
2: bye,
1: oldmast-head!
1: this?—green?
1: tiny
1: cracks.No
1: stains
1: nowbetween
1: matter’s.
1: oldtogether;
1: hulls,
1: liveflesh
1: shipsmade
1: vitalstuff
1: fathers.
1: beforeme,
1: pilot;
1: again?
1: where?
1: stairs?
1: Aye,aye,
1: told’st
1: OParsee;
1: short.
1: mast-head—keepa
1: We’ll
1: to-morrow,nay,
1: andtail.”He
1: loweredthrough
1: cloven
1: deck.In
1: shallop’sstern,
1: themate,—who
1: tackle-ropes
1: deck—and
1: pause.“Starbuck!”“Sir?”“For
1: Starbuck.”“Aye,
1: so.”“Some
1: missing,Starbuck!”“Truth,
1: truth.”“Some
1: tide;
1: theflood;—and
1: crested
1: comb,Starbuck.
1: old;—shake
1: man.”Their
1: met;
1: fastened;
1: glue.“Oh,
1: captain!—noble
1: heart—go
1: not—go
1: not!—see,
1: abrave
1: weeps;
1: persuasion
1: then!”“Lower
1: away!”—cried
1: crew!”In
1: stern.“The
1: sharks!
1: sharks!”
1: cabin-wi

1: awritten
1: explanation
1: youreceived
1: explanation.
1: entity
1: defective
1: elect
1: arefund.
1: electronically,
1: entityproviding
1: electronically
1: refund.
1: secondcopy
1: defective,
1: withoutfurther
1: problem.1.F.4.
1: forthin
1: ‘AS-IS’
1: WITH
1: OTHERWARRANTIES
2: ANY
1: KIND,
1: EXPRESS
1: IMPLIED,
1: INCLUDING
1: BUT
1: LIMITEDTO
1: WARRANTIES
1: MERCHANTIBILITY
1: FITNESS
1: PURPOSE.1.F.5.
1: disclaimers
1: impliedwarranties
3: limitation
1: types
1: damages.If
2: disclaimer
1: violatesthe
1: shallbe
1: interpreted
1: maximum
1: permittedby
1: invalidity
1: unenforceability
1: anyprovision
1: void
1: provisions.1.F.6.
1: INDEMNITY
1: indemnify
1: owner,
1: employee
1: anyoneproviding
1: production,promotion
3: distribution
1: works,harmless
1: liability,
1: fees,that
1: youdo
1: occur:
1: (a)
1: (b)
1: alteration,
1: modification,
1: additions
1: ordeletions
1: (c)
1: Defect
1: youcause.Section
4: Information
1: Mission
1: Gutenberg-tmProject
1: synonymous
1: ofe